In [1]:
import os
import random
import math
import h5py
import concurrent.futures
import matplotlib.pyplot as plt
font = {'size'   : 5}
plt.rc('font', **font)

import numpy as np
import pandas as pd
import pandas.api.types
from tqdm import tqdm
from glob import glob
from PIL import Image
import cv2
from io import BytesIO

import torch
from torch import nn
import torch.nn.functional as F
import lightning as L

import albumentations as albu
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, KFold, StratifiedGroupKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler


import timm
import wandb

import plotly.express as px
import plotly.graph_objects as go

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations
c:\Users\hoang\.conda\envs\bird\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lhkaay (lhklevi). Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

print('Using device:', device)

Using device: cuda


In [3]:
default_config= {
 'VERSION': 'v1.6',
 'DESCRIPTION': 'Add positive image samples from ISIC2020',
 'DATA_PATH': 'isic-2024-challenge',
 
 #-------Global-------#
 'IMAGE_SIZE': 256,
 'SEED': 24,
 #################################################
 
 #-------Image Augmentation-------#

 #################################################

 #-------Training hyperparameter-------#
 'BACKBONE': "tf_efficientnet_b3.in1k", # [ "eca_nfnet_l0" 256,"tf_efficientnet_b0.in1k" 256; "tf_efficientnetv2_s.in21k" 300;  "seresnext26t_32x4d.bt_in1k" 224; "tf_efficientnet_b3.ns_jft_in1k" 300; "resnet34.a1_in1k" 224; "efficientnet_b3.ra2_in1k" 288; "resnet50.a1_in1k" 224; "eca_nfnet_l0.ra2_in1k" 224]
 'EPOCHS': 10,
 'FOLD': 5,
 'BACTHSIZE': 64,
 'LOSS': 'BCE',
 #################################################

 'Avg OOF Valid Score':0
}

In [4]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(default_config['SEED'])

In [5]:
class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

#     # Equivalent code that uses sklearn's roc_auc_score
#     v_gt = abs(np.asarray(solution.values)-1)
#     v_pred = np.array([1.0 - x for x in submission.values])
#     max_fpr = abs(1-min_tpr)
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
#     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
#     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

# 1. Dataset

In [7]:
df_data = pd.read_csv(f"{default_config['DATA_PATH']}/train-metadata.csv")
# df_data = df_data.drop(df_data[df_data['target'] == 0].sample(frac=.995).index).reset_index(drop=True)


df_positive = df_data[df_data["target"] == 1].reset_index(drop=True)
df_negative = df_data[df_data["target"] == 0].reset_index(drop=True)

df_data = pd.concat([df_positive, df_negative.iloc[:df_positive.shape[0]*20, :]]).reset_index(drop=True)  # positive:negative = 1:20

hdf5_file = h5py.File(f"{default_config['DATA_PATH']}/train-image.hdf5", mode="r")

df_data = df_data[['isic_id', 'patient_id','target']]
df_data

C:\Users\hoang\AppData\Local\Temp\ipykernel_17512\3359923479.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(f"{default_config['DATA_PATH']}/train-metadata.csv")


,isic_id,patient_id,target
0,ISIC_0082829,IP_3249371,1
1,ISIC_0096034,IP_6723298,1
2,ISIC_0104229,IP_9057861,1
3,ISIC_0119495,IP_6856511,1
4,ISIC_0157834,IP_3927284,1
...,...,...,...
8248,ISIC_0267522,IP_9577633,0
8249,ISIC_0267560,IP_7746572,0
8250,ISIC_0267568,IP_0379091,0
8251,ISIC_0267594,IP_1433033,0


In [8]:
train_transform = albu.Compose([
        albu.Transpose(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.HorizontalFlip(p=0.5),
        albu.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.75),
        albu.OneOf([
            albu.MotionBlur(blur_limit=5),
            albu.MedianBlur(blur_limit=5),
            albu.GaussianBlur(blur_limit=5),
            albu.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),
        albu.OneOf([
            albu.OpticalDistortion(distort_limit=1.0),
            albu.GridDistortion(num_steps=5, distort_limit=1.),
            albu.ElasticTransform(alpha=3),
        ], p=0.7),
        albu.CLAHE(clip_limit=4.0, p=0.7),
        albu.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),

        albu.Resize(height=default_config['IMAGE_SIZE'], width=default_config['IMAGE_SIZE']),

        albu.CoarseDropout(max_height=int(default_config['IMAGE_SIZE'] * 0.0375), max_width=int(default_config['IMAGE_SIZE'] * 0.0375), p=0.7),
        
        albu.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2(p=1.0)])

valid_transform = albu.Compose([
        albu.Resize(height=default_config['IMAGE_SIZE'], width=default_config['IMAGE_SIZE']),
        albu.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2(p=1.0)])

C:\Users\hoang\AppData\Roaming\Python\Python39\site-packages\pydantic\main.py:176: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [9]:
class ISICDataset(torch.utils.data.Dataset):
    def __init__(self, hdf5_file, df_data, transform):
        self.df_data = df_data

        self.df_positive = df_data[df_data["target"] == 1].reset_index()
        self.df_negative = df_data[df_data["target"] == 0].reset_index()

        self.transform = transform
        self.fp_hdf = hdf5_file
    
    def __len__(self):
        return len(self.df_data)
    
    def __getitem__(self, index):
        if random.random() >=0.5:
            df = self.df_positive
        else:
            df = self.df_negative
        
        index = index % df.shape[0]
        sample_row = df.iloc[index]

        try:
            image = np.array(Image.open(BytesIO(self.fp_hdf[sample_row['isic_id']][()])))
        except:
            image = np.array(Image.open(f"isic-2024-challenge/ISIC2020/train-image/image/{sample_row['isic_id']}.jpg"))
        
        image = self.transform(image=image)['image']
        y = sample_row['target']
        
        return image, y

## 1.1 Display one batch in Dataset

In [10]:
def display_batch(malignant_only=False):
    tmp_df = df_2020data
    if malignant_only:
        tmp_df = df_data[df_data['target']==1]
    dataset = ISICDataset(hdf5_file=hdf5_file, df_data=tmp_df, transform=train_transform)
    fig = plt.figure(figsize=(20,10))
    
    img_index = np.random.randint(0, len(dataset)-1, 5*10)
    
    for i in range(len(img_index)):
        img, labels = dataset[img_index[i]]
        
        if isinstance(img, torch.Tensor):
            img = img.detach().numpy()
        ax = fig.add_subplot(5, 10, i + 1, xticks=[], yticks=[])
        # ax.imshow(np.transpose(img, (1,2,0)))
        ax.imshow(img)
        lbl = labels.item()
        ax.set_title(f'ID: {img_index[i]}; Target: {lbl}')

    plt.tight_layout()
    plt.show()


display_batch(malignant_only=False)

NameError: name 'df_2020data' is not defined

## 2. Model

In [11]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'
    
class ISICModel(nn.Module):
    def __init__(self, num_classes):
        super(ISICModel, self).__init__()

        self.backbone = timm.create_model(default_config['BACKBONE'], pretrained=True, num_classes=num_classes)

        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.backbone.global_pool = nn.Identity()

        self.globalpooling = GeM()
        self.dropout = nn.Dropout(p=0.3)
        self.linear = nn.Linear(in_features, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, ximage):
        x = self.backbone(ximage)
        x = self.globalpooling(x).flatten(1)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# model = ISICModel(num_classes=1).to(device)
# inputs = torch.randn(64, 3, 300, 300).to(device)

# with torch.autograd.profiler.profile(use_cuda=True) as prof:
#     outputs = model(inputs)

# print(prof.key_averages().table(sort_by="cuda_time_total"))

In [12]:
ssk = StratifiedGroupKFold(n_splits=default_config["FOLD"])
df_data['fold'] = 0
for fold, (train_idx, val_idx) in enumerate(ssk.split(df_data, df_data.target, df_data.patient_id)):
    df_data.loc[val_idx, 'fold'] = fold

In [13]:
df_2020data = pd.read_csv("isic-2024-challenge/ISIC2020/train-metadata.csv")

df_positive = df_2020data[df_2020data["target"] == 1].reset_index(drop=True)
df_negative = df_2020data[df_2020data["target"] == 0].reset_index(drop=True)
df_2020data = pd.concat([df_positive, df_negative.iloc[:df_positive.shape[0]*20, :]]).reset_index(drop=True)  # positive:negative = 1:20

# df_2020data = df_2020data[df_2020data['target']==1].reset_index(drop=True)[['isic_id', 'patient_id','target']]
df_2020data = df_2020data.reset_index(drop=True)[['isic_id', 'patient_id','target']]

df_2020data['fold'] = 0

In [14]:
if not os.path.exists(f"model/{default_config['VERSION']}"):
    os.makedirs(f"model/{default_config['VERSION']}")

# 3.Training loop

In [15]:
import gc
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [16]:
valid_score = []
for f in range(default_config["FOLD"]):
    run = wandb.init(project="ISIC2024", name=f"Model_Fold{f}_{default_config['VERSION']}", entity="lhklevi", config=default_config)
    wandb.alert(title=f"Start Model_Fold {f} !!!", text=f"Start Model_Fold {f} !!! [{default_config['DESCRIPTION']}]")
    # main loop of f-fold
    print('=================================================================================================')
    print(f"============================== Running training for fold {f} ==============================")
    
    train_df = df_data[df_data['fold'] != f].copy().reset_index(drop=True)
    train_df = pd.concat([train_df, df_2020data], ignore_index=True)
    valid_df = df_data[df_data['fold'] == f].copy().reset_index(drop=True)
    
    train_dataset = ISICDataset(hdf5_file, train_df, transform=train_transform)
    valid_dataset = ISICDataset(hdf5_file, valid_df, transform=valid_transform)

    print(f'Train Samples: {len(train_df)}')
#     display_batch(train_dataset)

    print(f'Valid Samples: {len(valid_df)}')
#     display_batch(valid_dataset)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=default_config["BACTHSIZE"], shuffle=True)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=default_config["BACTHSIZE"], shuffle=True)

    model = ISICModel(num_classes=1).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.00017, steps_per_epoch=len(train_dataloader), epochs=default_config["EPOCHS"], anneal_strategy='cos')
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=1)
    # scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr'])
    # Training loop
    for epoch in range(default_config["EPOCHS"]):
        model.train()
        for idx,batch in enumerate(train_dataloader):
            inputs, targets = batch

            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)

            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs).squeeze()

            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            print(f'Step {idx}/{len(train_dataloader)}, Loss: {loss.item():.4f}\r', end='', flush=True)
            wandb.log({"Learning Rate":  optimizer.param_groups[-1]['lr']}) #scheduler.get_last_lr()[0]
        model.eval()
        valid_step = []
        with torch.no_grad():
            for idx, batch in enumerate(valid_dataloader):
                inputs, targets = batch
                inputs = inputs.to(device, dtype=torch.float)

                outputs = model(inputs).squeeze()
                valid_step.append({"logits": outputs, "targets": targets.to(torch.float)})

            output_val = torch.cat([x['logits'] for x in valid_step], dim=0).cpu().detach()
            target_val = torch.cat([x['targets'] for x in valid_step], dim=0).cpu().detach()

            val_loss = criterion(output_val, target_val)

            gt_df = pd.DataFrame(target_val.numpy().astype(np.float32), columns=['target'])
            pred_df = pd.DataFrame(output_val.numpy().astype(np.float32), columns=['target'])
            
            gt_df['isic_id'] = [f'id_{i}' for i in range(len(gt_df))]
            pred_df['isic_id'] = [f'id_{i}' for i in range(len(pred_df))]
            val_roc_auc = score(gt_df, pred_df, row_id_column_name='isic_id')
            
            scheduler.step(val_roc_auc)

        print(f"Epoch {epoch+1}/{default_config['EPOCHS']}, train_loss: {loss.item():.4f}, valid_loss: {val_loss:.4f}, valid_score: {val_roc_auc:.4f} lr: {scheduler.get_last_lr()}")
        wandb.log({"Training Loss": loss.item(),"Valid Loss": val_loss, "Valid Score": val_roc_auc})
        gc.collect()

    valid_score.append(val_roc_auc)
    if f==4:
        wandb.config['Avg OOF Valid Score'] = sum(valid_score) / len(valid_score)
    
    torch.save(model.state_dict(), f"model/{default_config['VERSION']}/Model_Fold{f}.pt")
    wandb.save(f"model/{default_config['VERSION']}/Model_Fold{f}.pt")
    if f==4:
        wandb.alert(title=f"Done All Fold {f} !!!", text=f"Done All Fold {f} OOF Score: {sum(valid_score) / len(valid_score)}!!! [{default_config['DESCRIPTION']}]")
    else:
        wandb.alert(title=f"Done Fold {f} !!!", text=f"Done Fold {f} !!! [{default_config['DESCRIPTION']}]")
    run.finish()

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/tf_efficientnet_b3.in1k)


============================== Running training for fold 0 ==============================
Train Samples: 18866
Valid Samples: 1651


INFO:timm.models._hub:[timm/tf_efficientnet_b3.in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


Epoch 1/10, train_loss: 0.4635, valid_loss: 0.3493, valid_score: 0.1525 lr: [0.0001]
Epoch 2/10, train_loss: 0.2904, valid_loss: 0.3520, valid_score: 0.1489 lr: [0.0001]
Epoch 3/10, train_loss: 0.3481, valid_loss: 0.3381, valid_score: 0.1551 lr: [0.0001]
Epoch 4/10, train_loss: 0.2362, valid_loss: 0.3681, valid_score: 0.1459 lr: [0.0001]
Epoch 5/10, train_loss: 0.3017, valid_loss: 0.3767, valid_score: 0.1502 lr: [1e-05]
Epoch 6/10, train_loss: 0.2954, valid_loss: 0.3819, valid_score: 0.1493 lr: [1e-05]
Epoch 7/10, train_loss: 0.3719, valid_loss: 0.3493, valid_score: 0.1547 lr: [1.0000000000000002e-06]
Epoch 8/10, train_loss: 0.2798, valid_loss: 0.3472, valid_score: 0.1554 lr: [1.0000000000000002e-06]
Epoch 9/10, train_loss: 0.2006, valid_loss: 0.4012, valid_score: 0.1467 lr: [1.0000000000000002e-06]
Epoch 10/10, train_loss: 0.2112, valid_loss: 0.3673, valid_score: 0.1496 lr: [1.0000000000000002e-07]


Learning Rate,████████████████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Training Loss,█▃▅▂▄▄▆▃▁▁
Valid Loss,▂▃▁▄▅▆▂▂█▄
Valid Score,▆▃█▁▄▄██▂▄
Learning Rate,0.0
Training Loss,0.21121
Valid Loss,0.36732
Valid Score,0.14955


...

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/tf_efficientnet_b3.in1k)


============================== Running training for fold 4 ==============================
Train Samples: 18868
Valid Samples: 1649


INFO:timm.models._hub:[timm/tf_efficientnet_b3.in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


Epoch 1/10, train_loss: 0.3498, valid_loss: 0.3815, valid_score: 0.1300 lr: [0.0001]
Epoch 2/10, train_loss: 0.4244, valid_loss: 0.3633, valid_score: 0.1394 lr: [0.0001]
Epoch 3/10, train_loss: 0.4345, valid_loss: 0.3496, valid_score: 0.1425 lr: [0.0001]
Epoch 4/10, train_loss: 0.3760, valid_loss: 0.4378, valid_score: 0.1399 lr: [0.0001]
Epoch 5/10, train_loss: 0.4742, valid_loss: 0.3591, valid_score: 0.1414 lr: [1e-05]
Epoch 6/10, train_loss: 0.2751, valid_loss: 0.3889, valid_score: 0.1389 lr: [1e-05]
Epoch 7/10, train_loss: 0.1848, valid_loss: 0.3869, valid_score: 0.1361 lr: [1.0000000000000002e-06]
Epoch 8/10, train_loss: 0.3339, valid_loss: 0.3927, valid_score: 0.1349 lr: [1.0000000000000002e-06]
Epoch 9/10, train_loss: 0.5064, valid_loss: 0.3914, valid_score: 0.1371 lr: [1.0000000000000002e-07]
Epoch 10/10, train_loss: 0.2965, valid_loss: 0.4416, valid_score: 0.1260 lr: [1.0000000000000002e-07]


Learning Rate,████████████████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Training Loss,▅▆▆▅▇▃▁▄█▃
Valid Loss,▃▂▁█▂▄▄▄▄█
Valid Score,▃▇█▇█▆▅▅▆▁
Learning Rate,0.0
Training Loss,0.29653
Valid Loss,0.44158
Valid Score,0.12599


# Evaluation